# Interpolate a meridional section of CROCO outputs to regular depths

This notebook, we show:

* how to compute the depths from s-coordinates,
* how to easily find the name of variables and coordinates,
* how to interpolate a 3D field with varying depths to regular depths.

## Inits

Import needed modules.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
import xoa
from xoa.regrid import regrid1d

xr.set_options(display_style="text");

Register the [decode_sigma](file:///local/home/sraynaud/Src/xoa/doc/_build/html/api/xoa.accessors.SigmaAccessor.html#xoa.accessors.SigmaAccessor) accessor.

In [ ]:
xoa.register_accessors(decode_sigma=True)

The [xoa](file:///local/home/sraynaud/Src/xoa/doc/_build/html/api/xoa.accessors.XoaDatasetAccessor.html#xoa.accessors.XoaDatasetAccessor) accessor is also registered by default, and give access to most of the fonctionalities of the other accessors.

## Read the model
This sample is a meridional extraction of a full 3D CROCO output.

In [ ]:
ds = xoa.open_data_sample("croco.south-africa.meridional.nc")
ds

## Compute depths from s-coordinates
Decode the dataset according to the CF conventions:

1. Find sigma terms
2. Compute depths
3. Assign depths as coordinates

Note that the [decode_sigma](file:///local/home/sraynaud/Src/xoa/doc/_build/html/api/xoa.accessors.SigmaAccessor.html#xoa.accessors.SigmaAccessor) accessor calls the [xoa.sigma.decode_cf_sigma](file:///local/home/sraynaud/Src/xoa/doc/_build/html/api/xoa.sigma.decode_cf_sigma.html#xoa.sigma.decode_cf_sigma) function.

In [ ]:
ds = ds.decode_sigma()
ds.depth

## Find coordinate names from CF conventions
The `depth` was assigned as coordinates at the previous stage.
We use the [xoa](file:///local/home/sraynaud/Src/xoa/doc/_build/html/api/xoa.accessors.XoaDatasetAccessor.html#xoa.accessors.XoaDatasetAccessor) data array accessor to easily access the temperature, latitude and depth arrays.

In [ ]:
temp = ds.xoa.temp.squeeze()
lat_name = temp.xoa.lat.name

## Interpolate at regular depths

We interpolate the temperature array from irregular to regular depths.

Let's create the output depths.

In [ ]:
depth = xr.DataArray(np.linspace(ds.depth.min(), ds.depth.max(), 100), 
                     name="depth", dims="depth")

Let's interpolate the temperature.

In [ ]:
tempz = regrid1d(temp, depth)

## Plots
Make a basic comparison plots.

In [ ]:
fig, axs = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(10, 4))
kw = dict(levels=np.arange(0, 23))
temp.plot.contourf(lat_name, "depth", cmap="cmo.thermal", ax=axs[0], **kw)
temp.plot.contour(lat_name, "depth", colors='w', linewidths=.3, ax=axs[0], **kw)
tempz.plot.contourf(lat_name, "depth", cmap="cmo.thermal", ax=axs[1], **kw)
tempz.plot.contour(lat_name, "depth", colors='w', linewidths=.3, ax=axs[1], **kw);

Et voilà!